This is a helper script to replace the named drs uris with numbered drs uris in the fields "cram_drs_uri", "crai_drs_uri", "bundle_drs_uri" in the data-connect starter kit v0.1.1 default dataset for one_thousand_genomes_sample table

In [4]:
import sqlite3
import json

db_name = "../resources/data-connect/db/data-connect.db"
connection = sqlite3.connect(db_name)


# Get mapping of existing drs_ids (ids with file name in it) and required drs_ids (opaque ids - md5)
with open("../drs_map.json") as f:
    drs_map = json.load(f)
    
# Helper function to replace drs file names with md5 ids in ['cram_drs_uri', 'crai_drs_uri', 'bundle_drs_uri'] fields
def fix_row(line):
    key_list = ['cram_drs_uri', 'crai_drs_uri', 'bundle_drs_uri']
    sample = json.loads(line)
    for k in key_list:
        # get the part after the last /
        drs_id = sample[k].split('/')[-1]
        sample[k] =  f"drs://localhost:5000/{drs_map[drs_id]}"
    #print(json.dumps(sample, indent=3))
    return sample

In [5]:
# Connect to database and get all existing rows
cursor = connection.cursor()
rows = cursor.execute(
    "SELECT id, json_data FROM one_thousand_genomes_sample",

).fetchall()
id = str(rows[0][0])
print(rows[0][1])
print("Number of rows:" + str(len(rows)))

{"sample_name": "HG01965", "sex": "female", "biosample_id": "SAME1839145", "population_code": "PEL", "population_name": "Peruvian", "superpopulation_code": "AMR", "superpopulation_name": "American Ancestry", "population_elastic_id": "PEL", "data_collections": "1000 Genomes on GRCh38,1000 Genomes 30x on GRCh38,1000 Genomes phase 3 release", "cram_drs_uri": "drs://localhost:5000/HG01965.1000genomes.lowcov.downsampled.cram", "crai_drs_uri": "drs://localhost:5000/HG01965.1000genomes.lowcov.downsampled.crai", "bundle_drs_uri": "drs://localhost:5000/HG01965.1000genomes.lowcov.downsampled.bundle"}
Number of rows:200


In [6]:
jstring = ""
for r in rows:
    id =str(r[0])
    new_json = fix_row(r[1])
    #print(new_json)
    jstring = jstring + "(\'"+json.dumps(new_json)+"\'),\n"
connection.commit()

In [7]:
insert_statement = "Insert into one_thousand_genomes_sample (json_data) VALUES " + jstring.strip().rstrip(',')+";"

In [12]:
text_file = open("./add-dev-dataset.sql", "a")
text_file.write(insert_statement)
text_file.close()